# Crawler
 
lets call crawler and fetch some documents

In [1]:
!root=~/PycharmProjects/

In [2]:
!cd Crawler

In [3]:
from Crawler.crawler import Crawler

crawler = Crawler(downloaded=False)
print(crawler.downloaded)
crawler.get_collection()


False
file reut2-002 docs are being processed and extracted to ./.misc/docs/reut2-002


In [4]:
crawler.retrieve_docs() # retrieve random doc from the docs collection :3 

"./.misc/docs/reut2-002/879.txt\n\n 6-MAR-1987 16:33:01.61\n\nusasouth-korea\n\n\n\n\n \n\x05\x05\x05F\n\x16\x16\x01f0575\x1freute\nu f AM-HAWKS   03-06 0089\n\x02\nU.S. TO SELL ELECTRONIC EQUIPMENT TO S. KOREA\n    WASHINGTON, March 6 - The United States plans to sell 84\nmln dlrs in electronic equipment to South Korea to upgrade that\ncountry's Hawk anti-aircraft missiles, the Reagan\nadministration announced.\n    The sale of 28 modification kits to upgrade the I-Hawk\n(Improved Hawk) missiles already in South Korea is expected to\nbecome official in 30 days without objection from Congress.\n    Prime contractor on the sale is the Raytheon Corp <RTN>,\nwhich builds the Hawk medium-range missiles.\n Reuter\n\x03\n"

In [5]:
from Indexer.indexer import Indexer
indexer = Indexer()

In [47]:
doc = crawler.retrieve_docs()
path = doc.split('\n')[0]
doc = '\n'.join(doc[1:])
doc = path,doc
indexer(doc)

In [48]:
indexer.search("he*")

he*
['he*']
Retrieving data from db
Retrieving data from db
Retrieving data from db
Retrieving data from db
Retrieving data from db
Query expression is ::   ($hearing$ || $headline$ || $headlined$ || $heller$ || $heatusa$ || $helpedkeep$ || $held$ || $help$ || $heating$) 


(' ($hearing$ || $headline$ || $headlined$ || $heller$ || $heatusa$ || $helpedkeep$ || $held$ || $help$ || $heating$) ',
 ['./.misc/docs/reut2-002/914.txt',
  './.misc/docs/reut2-002/725.txt',
  './.misc/docs/reut2-002/534.txt',
  './.misc/docs/reut2-002/804.txt',
  './.misc/docs/reut2-002/364.txt',
  './.misc/docs/reut2-002/634.txt',
  './.misc/docs/reut2-002/643.txt',
  './.misc/docs/reut2-002/696.txt'])

In [49]:
indexer.search("in*y")

in*y
['in*y']
Retrieving data from db
Query expression is ::   ($industry$ || $injanuary$) 


(' ($industry$ || $injanuary$) ',
 ['./.misc/docs/reut2-002/48.txt', './.misc/docs/reut2-002/995.txt'])

In [50]:
indexer.parser.preprocess("in*y+rs*g")

['$iny$', '$rsg$']

In [10]:
max([len(indexer.aux_index[key]) for key in indexer.aux_index.keys()])

1